# 章 データの読み込み、書き出しとファイル形式 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(20, 6))
np.set_printoptions(precision=4, suppress=True)

np.random.seed(12345)

## テキスト形式データの読み書き
p181
- pandasにはテーブル形式のデータをDataFrameオブジェクトとして読み込む関数が豊富にある。
  [表6-1](https://pandas.pydata.org/pandas-docs/stable/api.html)参照

p182
- 読み込み関数のいくつか（例：pandas.read_csv）では型推論が行われる為、データの型を明示的に指定しなくとも良い場合もある。


In [2]:
# OSコマンドでテキストファイルの中身を確認（以下はmacの場合）
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [3]:
# ファイルがカンマ区切りなので、read_csvでデータフレーム に変換
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
# sepオプションで、区切り文字を指定する事でread_tableでも変換可能
pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
# ヘッダ行がないファイルの中身を確認
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [6]:
# headerオプションで、pandasにデフォルトの列名を割り当てて貰う。
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
# namesオプションで自前でヘッダ行を付与
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


p184
- 列値をヘッダー名に割り当てるには__`index_col`__オプションを使用する。

In [8]:
names = ['a', 'b', 'c', 'd', 'message']

# namesリストをヘッダー名に使用し、message列値をインデックスにする。
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


p184
- 階層型インデックスを作成したい場合は、__`index_col`__オプションに列の番号か、列名のリストを与える。

In [9]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [10]:
# key1, key2で階層型インデックスを持ったDataFrameを作成する。
parsed = pd.read_csv('examples/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

p185
- 正規表現を用いて、区切り文字をしていることも可能。
- 一つ以上の空白の正規表現は__`\s+`__

In [11]:
# 区切り文字に空白が使用されている場合
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [12]:
# １つ以上の空白を区切り文字として読み込む
result = pd.read_table('examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


1行の列数が、他の行よりも少ないので、1行目はDataFrameのヘッダと推測する。  

p185
- read_csvのオプションは [表6−2](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html#pandas.read_csv)参照。

In [13]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [14]:
# skiprowsオプションで、1行目、３行目、4行目を読み飛ばす。
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


p186
- 空文字やNA、Null で表現されている箇所は常に欠損値 (Nan) 扱い。

In [15]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

2行目位に`NA`、3行目に空文字がある。

In [16]:
# NAの部分が欠損値 NaN扱い
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [17]:
# 欠損値の確認
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


p186
- na_valuesオプションで欠損値と見なす文字列を指定する事ができる。
- 指定の仕方はリストかセット。

In [18]:
# worldという文字列を欠損値扱いにする。
result = pd.read_csv('examples/ex5.csv', na_values=['world'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,NaN
2,three,9,10,11.0,12,foo


p186
- 列毎に欠損値の値を指定する事も可能。

In [19]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}

# message列はfooを、something列はtwoという文字列を欠損値扱いにする。
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### テキストファイルを少しづつ読み込む

p188
- pandasの表示オプションは、__`pd.options.display.***`__で行う。

In [20]:
# 表示行を10行に制限する。
pd.options.display.max_rows = 10

In [21]:
# １万行あるファイルを表示する。
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


p188
- 最初の数行だけ読み込みたい場合は、__`nrows`__オプションを使う。

In [22]:
# 最初の五行だけ読み込む
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


p188
- 一度に読み込む行数を指定する場合は、__`chunksize`__オプションを使う。

In [23]:
# １度に読み込む行数を1000行までとする。
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

__`chunksize`__オプション作成した `pandas.io.parsers.TextFileReader` オブジェクトは繰り返し処理の中で使用する事ができる。

In [24]:
# chunksizeで TextFileReader オブジェクト作成
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

# key列の値の個数を格納するシリーズ（インデックスがkey列の値）
tot = pd.Series([])

# 繰り返し処理（key列の
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
     ...  
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
Length: 36, dtype: float64

In [25]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### テキスト形式でのデータの書き出し
p189
- 区切り文字を指定してファイルに出力する事が出来る。
- __`to_csv`__メソッドでカンマ区切りのファイルに出力できる。

In [26]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [27]:
# dataをcsv形式（カンマ区切り）でout.csvとして出力
data.to_csv('examples/out.csv')
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [28]:
import sys

# dataを標準出力に出力。区切り文字は｜
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


p190
- 欠損値は空文字として出力。
- 欠損値を空文字以外の文字で出力したい場合は、__`na_rep`__オプションで指定する。

In [29]:
# 欠損値をNULLと表記してdataを標準出力に出力。
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


p190
- 行ラベルと、列ラベルは無効化する事も可能。

In [30]:
# 行ラベルと、列ラベルは無効化
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


p190
- 一部の列だけ、指定した順序で出力する事が可能。

In [31]:
# a,b,c,という順序で出力（それ以外の列は出力しない）
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


p190
- シリーズも __`to_csv`__ メソッドで出力可能。

In [32]:
# 2000/01/01 - 2000/07/01 までの日付データを作成
dates = pd.date_range('1/1/2000', periods=7)
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')

In [33]:
# datesの要素を行ラベルにしてシリーズを作成
ts = pd.Series(np.arange(7), index=dates)
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int64

In [34]:
# シリーズをtseries.csvファイルに出力
ts.to_csv('examples/tseries.csv')
!cat examples/tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### 区切り文字で区切られた形式を操作する

In [35]:
# 区切り文字が(",")、囲み文字(")のファイル
!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [36]:
import csv

with open('examples/ex7.csv') as f:

    # 区切り文字が１文字の場合、組み込みのcsvモジュールが使える
    reader = csv.reader(f)

    # 囲み文字を取り除いたタプルを戻す
    for line in reader:
        print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


__`csv.reader`__から戻されたreaderに対して繰り返し処理を行うと、囲み文字が全て取り除かれたタプルが返される。

In [37]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

In [38]:
# 1行目をヘッダーとして使用する。
header, values = lines[0], lines[1:]

print("header:{} \nvalues:{}".format(header, values))

header:['a', 'b', 'c'] 
values:[['1', '2', '3'], ['1', '2', '3']]


In [39]:
# zip(*values)で、valuesの行と列を入れ替える
for v in zip(*values):
    print(v)

('1', '1')
('2', '2')
('3', '3')


In [40]:
# ヘッダ行をキーとしたディクショナリを作成
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

p192
- csv,Dialectのサブクラスを定義する事で、区切り文字、引用符（囲み文字）、行終端記号（改行文字）などをカスタマイズする事ができる。
- csv.Dialectのサブクラスを定義する代わりに、オプションを直接パラメータとして渡す事も可能。

In [41]:
# csv.Dialectのサブクラスを定義
# lineterminator：行終端記号
# delimiter:区切り文字
# quotechar:引用符
# quoting:引用符の使用ルール（csv.QUOTE_MINIMALは、特殊文字を含んだフィールドのみを囲む）
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [42]:
f = open('examples/ex7.csv')

# クラスを指定してCSVを読み込む
reader = csv.reader(f, dialect=my_dialect)

In [43]:
f = open('examples/ex7.csv')

# csv.Dialectのサブクラスを定義する代わりに、オプションを直接パラメータとして渡せる
reader = csv.reader(f, delimiter='|')

In [44]:
f = open('examples/ex7.csv')

# 自分で定義したCSVの方言を用いて出力
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

In [45]:
# 作成したファイルの中身を確認(区切り文字= ; )
!cat mydata.csv

one;two;three
1;2;3
4;5;6
7;8;9


### JSON Data
p193
- JSONはHTTPリクエストでデータをやり取りする標準的なデータ形式。
- CSVより遥かに表現力の自由度が高い。
- null値は__`null`__で表現する。
- リストの最終要素にコンマ(,)は記述できない。
- 組み込みの__`json`__モジュールでJSON形式のデータをPython形式に変換できす。

In [46]:
# JSONの例
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

print(obj)


{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}



In [47]:
# 組み込みライブラリを使用してJSON形式のデータをPython形式(ディクショナリ）に変換
import json
result = json.loads(obj)

print("### type(result) : {}".format(type(result)))

result

### type(result) : <class 'dict'>


{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 30, 'name': 'Scott', 'pets': ['Zeus', 'Zuko']},
  {'age': 38, 'name': 'Katie', 'pets': ['Sixes', 'Stache', 'Cisco']}]}

p194
- __`dump`__メソッドでPythonオブジェクトをJSONに変換。

In [48]:
# PythonオブジェクトをJSONに変換
asjson = json.dumps(result)

print("### type(asjson) : {}".format(type(asjson)))

asjson

### type(asjson) : <class 'str'>


'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [49]:
# ディクショナリ(result)の siblings の name と age をDataFrameとして抽出
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


p194
- __`pandas.read_json`__メソッドでも、JSONデータを DataFrame や Seriaes に変換可能。
- __`pandas.read_json`__メソッドのデフォルトでは、各オブジェクト（下記の例では、a,b,c）が DataFrame の行になる。
- データフレーム からJSONに変換するには__`to_json`__メソッドを使う。


In [50]:
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [51]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [52]:
# データフレーム をJSON形式に変換
print(data.to_json())
print(data.to_json(orient='records'))

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}
[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### XML と HTML: ウェブスクレイピング
p195
- Pythonは、HTML/XML形式のデータの読み書きライブラリが豊富。  
　(例：lxml、Beautiful Soup、 html5lib など)
- lxmlは高速だが、規格に従っていないHTMLやXMLを扱う場合は、Beautiful Soup、 html5libの方が便利。
- pandasでは、__`read_html`__を提供している。
- __`read_html`__は、lxml、Beautiful Soup の何れかを使用して解析している。
- __`read_html`__のデフォルトは、table形式のデータを全て探索し、DataFrameのリストとして抽出する。  
  （[例：examples/fdic_failed_bank_list.html](examples/fdic_failed_bank_list.html))

In [53]:
# dic_failed_bank_list.htmlから全てのテーブルを探索し、データフレーム のリストとして抽出
tables = pd.read_html('examples/fdic_failed_bank_list.html')
len(tables)

1

In [54]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


#### lxml.objectifyを使ったXMLの読み込み
p196
- [例：datasets/mta_perf/Performance_MNR.xml](datasets/mta_perf/Performance_MNR.xml)

In [55]:
from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'

# lxml.objectifyを使用してPerformance_MNR.xmlファイルを解析
parsed = objectify.parse(open(path))

# XMLファイルのルートノード（PERFORMANCE）を取得
root = parsed.getroot()
root

<Element PERFORMANCE at 0x1117cdc08>

In [56]:
# 解析結果を格納
data = []

# XMLから下記のフィールドを除外
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

# PERFORMANCE配下のINDICATOR要素を繰り返し処理
for elt in root.INDICATOR:
    el_data = {}
    
    # INDICATOR配下の子要素を繰り返し処理   
    for child in elt.getchildren():
        
        # 除外フィールドの判定         
        if child.tag in skip_fields:
            continue
        
        # 除外対象でなければ、ディクショナリに格納
        el_data[child.tag] = child.pyval
    data.append(el_data)

perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


In [57]:
from io import StringIO

# 擬似XML文書（本例では a要素の herf属性 の取得を試す）
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [58]:
root
root.get('href')

'http://www.google.com'

In [59]:
root.text

'Google'

## Binary Data Formats

In [60]:
frame = pd.read_csv('examples/ex1.csv')
frame
frame.to_pickle('examples/frame_pickle')

In [61]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [62]:
!rm examples/frame_pickle

### Using HDF5 Format

In [63]:
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [64]:
store['obj1']

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781
...,...
95,0.795253
96,0.118110
97,-0.748532
98,0.584970


In [65]:
store.put('obj2', frame, format='table')
store.select('obj2', where=['index >= 10 and index <= 15'])
store.close()

In [66]:
frame.to_hdf('mydata.h5', 'obj3', format='table')
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781


In [67]:
os.remove('mydata.h5')

NameError: name 'os' is not defined

### Reading Microsoft Excel Files

In [ ]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [ ]:
pd.read_excel(xlsx, 'Sheet1')

In [ ]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

In [ ]:
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [ ]:
frame.to_excel('examples/ex2.xlsx')

In [ ]:
!rm examples/ex2.xlsx

## Interacting with Web APIs

In [ ]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

In [ ]:
data = resp.json()
data[0]['title']

In [ ]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

## Interacting with Databases

In [ ]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

In [ ]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [ ]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

In [ ]:
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

In [ ]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

In [ ]:
!rm mydata.sqlite

## Conclusion